<a href="https://colab.research.google.com/github/sssanghn/AI-Emotion-Diary/blob/main/KoBERT_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KoBERT 감정 다중분류모델

> KoBERT를 이용하여 한국어 문장을 여러 클래스로 분류하는 모델을 만들어 보려고 한다.
</br>
공포, 놀람, 분노, 슬픔, 중립, 기쁨, 혐오와 같은 감정이 느껴지는 짧은 대화 텍스트를 각각 어떠한 감정의 텍스트인지 분류하는 모델을 만드는 것이다. </br>
AIHUB에서 가져온 데이터셋으로 KoBERT 모델을 FineTuning 하여 </br>
최종적으로 AI 감정일기 어플리케이션에서 작성한 일기 텍스트에서 감정을 추출하기 위해 프로젝트를 진행하였다.

**KoBERT란?**

> BERT는 약 33억 개의 단어로 pretrain 되어 있는 기계번역 모델이다. </br>
하지만 외국에서 만든 것이다 보니 영어에 대해서는 정확도는 높고, 한국어에 대해서는 정확도가 떨어진다.</br>
따라서 BERT 모델을 한국어에도 잘 활용할 수 있도록 만들어진 것이 KoBERT이다. </br>
KoBERT는 BERT 모델에서 한국어 데이터를 추가로 학습시킨 모델로, 한국어 위키에서 5백만개의 문장과 5천4백만개의 단어를 학습시킨 모델이다. </br>
따라서 한국어 데이터에 대해서도 높은 정확도를 낼 수 있다고 한다. </br>
한국어의 불규칙한 언어 변화의 특성을 반영하기 위해 데이터 기반 토큰화 (Tokenization) 기법을 적용하여 기존 대비 27%의 토큰만으로 2.6% 이상의 성능 향상을 이끌어 낸 모델이다. </br>

# 1. 프로젝트 설명

**KoBERT 파인튜닝**
> BERT 모델은 자신의 사용 목적에 따라 파인튜닝(FineTuning)이 가능하다. </br>
따라서 Output layer만 추가로 달아주면 원하는 결과를 출력해내는 기계번역 모델을 만들어 낼 수 있다. </br>
이 점을 활용하여 KoBERT 모델을 FineTuning하여 최종적으로 일기 텍스트에서 감정을 추출해내는 모델을 새로 만들어 보려고 한다.

# 2. 데이터 수집

**데이터 수집**
> 이번 프로젝트를 위해 사용한 한국어 대화 문장 데이터셋은 총 2개이다. </br>
</br>
1. 한국어 감정 정보가 포함된 단발성 대화 데이터셋
* SNS 글 및 온라인 댓글에 대한 웹 크롤링을 실시하여 선정된 AIHUB의 공공 데이터셋
* 약 3만 9천건의 데이터 셋 (기쁨, 슬픔, 놀람, 분노, 공포, 혐오, 중립의 7개 감정)
</br>
</br>
2. 감성 대화 말뭉치
* 한국어 감성 데이터셋을 상세한 감정 라벨링(감정 대분류와 소분류로 구분)과 함께 제공하는 AIHub의 공공 데이터셋
* 약 4만 건의 데이터 셋 (기쁨, 슬픔, 당황, 분노, 불안, 상처의 6개 감정)
* 데이터 병합을 위해 감정 대분류와 대화 문장을 남기고, 감정 소분류는 누락

# Colab 환경 설정

> 모델을 구현하기에 앞서 코랩 환경설정을 진행하려고 한다. </br>
해당 사양은 KoBERT 오픈소스 내 requirements.txt를 참고하였다.

In [30]:
# Colab 환경 설정
# requirements : https://github.com/SKTBrain/KoBERT/blob/master/kobert_hf/requirements.txt
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install sentencepiece
!pip install transformers
!pip install torch

> 다음으로 깃허브 내 파일을 Colab으로 가져온다.

In [31]:
# https://github.com/SKTBrain/KoBERT의 파일들을 Colab으로 다운로드
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-vq7493mj/kobert-tokenizer_21bd9405ab294d81a09de7c72e627197
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-vq7493mj/kobert-tokenizer_21bd9405ab294d81a09de7c72e627197
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


> 그 다음으로는 해당 라이브러리들을 import 해준다.

In [32]:
# KoBERT
from kobert_tokenizer import KoBERTTokenizer

# Transformers
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

# Setting Library
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

In [18]:
device = torch.device("cuda:0")

In [34]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
tok = tokenizer.tokenize

# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [35]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):

        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [36]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

> BERT 데이터셋을 구성하는 라벨의 개수를 num_classes로 넣어주거나 모델을 통해 직접 추론할 수도 있다.

In [37]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [38]:
def predict(sentence):
    dataset = [[sentence, '0']]
    test = BERTDataset(dataset, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=2)
    model.eval()
    answer = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        for logits in out:
            logits = logits.detach().cpu().numpy()
            answer = np.argmax(logits)
    return answer

> 주어진 문장이 현재 학습이 완료된 모델 내에서 어떤 라벨과 argmax인지 판단하고 추론된 결과를 리턴하는 함수

In [45]:
dataset_data1 = pd.read_excel('/content/drive/MyDrive/dataset/한국어_단발성_대화_데이터셋.xlsx')
dataset_data2 = pd.read_excel('/content/drive/MyDrive/dataset/감성대화말뭉치Training.xlsx')
dataset_data3 = pd.read_excel('/content/drive/MyDrive/dataset/감성대화말뭉치Validation.xlsx')


len(dataset_data1)
dataset_data1.sample(n=10)

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
9507,어휴 이런 나라에서 애 키우는 사람들이 정말 대단하신거다,놀람,NaN,NaN,NaN,NaN,NaN
27616,글정리같은거랄까 그냥 깔끔한걸 좋아해요 ..,행복,NaN,NaN,NaN,NaN,NaN
9516,25년ㄷㄷㄷㄷ 짱이다,놀람,NaN,NaN,NaN,NaN,NaN
38356,정말 대책도 없고 할말을 잃었다,혐오,NaN,NaN,NaN,NaN,NaN
1843,지금은 다른 부대 배치가 된건지 모르는 상황입니다...,공포,NaN,NaN,NaN,NaN,NaN
29351,나의 우상 정지훈형님,행복,NaN,NaN,NaN,NaN,NaN
33243,인간들이란.,혐오,NaN,NaN,NaN,NaN,NaN
13793,황교안 겉으로는 대통령 행세하면서 뒷구녕으로는 박근혜&최순실 비위맞추고 보호하고 있...,분노,NaN,NaN,NaN,NaN,NaN
28702,와 투수 정말 잘하더라..,행복,NaN,NaN,NaN,NaN,NaN
7437,패자부활전으로 다시올라왔을땐 읭?,놀람,NaN,NaN,NaN,NaN,NaN


In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

In [ ]:
torch.save(model, f'/content/drive/MyDrive/Colab Notebooks/SentimentAnalysisKOBert.pt')
torch.save(model.state_dict(), f'/content/drive/MyDrive/Colab Notebooks/SentimentAnalysisKOBert_StateDict.pt')